In [1]:
%matplotlib inline
!nvidia-smi

Thu Nov 30 00:59:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/VikashKumarShaw1994/DNN_invariant_time-rescaling.git DNN_invariant_sithcon

fatal: destination path 'DNN_invariant_sithcon' already exists and is not an empty directory.


In [3]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
#from deepsith import DeepSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

<class 'torch.cuda.FloatTensor'>


In [4]:
rosicrucian_cipher = {
    'A': '☉',
    'B': '☿',
    'C': '♀',
    'D': '☽☿',
    'E': '☉♀',
    'F': '☽☽',
    'G': '☽♀',
    'H': '☽☉',
    'I': '☉☉',
    'J': '☿♀',
    'K': '☽☉☽',
    'L': '☽☽♀',
    'M': '☽☿☽',
    'N': '☽',
    'O': '☉☉',
    'P': '☉☽',
    'Q': '☽☉',
    'R': '☉☉☽',
    'S': '☉☉☉',
    'T': '☽☽☽',
    'U': '☉☉♀',
    'V': '☉☉☽☽',
    'W': '☽☽☽☽',
    'X': '☽☉☉☉',
    'Y': '☽☉☽☉',
    'Z': '☽☉☉☉☉',
    '0': '♀☽☽☽☽',
    '1': '☉☽☽☽☽',
    '2': '☉☉☽☽☽',
    '3': '☉☉☉☽☽',
    '4': '☉☉☉☉☽',
    '5': '☉☉☉☉☉',
    '6': '♀☉☉☉☉',
    '7': '☽♀☉☉☉',
    '8': '☽☽♀☉☉',
    '9': '☽☽☽♀☉',
    ',': '♀☽',
    '.': '☉☽☉☽',
    '(': '☽☉☉☉☽',
    ')': '☽☉☉☉☉',
    '/': '☉☽☽☉',
    '?': '☽☽♀☽☉',
    '-': '☽☉☉☽☉',
}



message = "CRYPTO"
binary_representation = ''.join([rosicrucian_cipher[char] for char in message])

print(binary_representation)


♀☉☉☽☽☉☽☉☉☽☽☽☽☉☉


In [7]:
rosicrucian_to_binary = {
    '☉': '00',
    '☿': '01',
    '♀': '10',
    '☽': '11',
}




print(binary_representation)

def rosicruciantobinary(rosicrucian_message):
    binary_representation = ''.join([rosicrucian_to_binary[char] for char in rosicrucian_message])

    return binary_representation

rosicrucian_cipher_dict={}
for brk,brv in rosicrucian_cipher.items():
    rosicrucian_cipher_dict[brk]=rosicruciantobinary(brv)
print(rosicrucian_cipher_dict)

♀☉☉☽☽☉☽☉☉☽☽☽☽☉☉
{'A': '00', 'B': '01', 'C': '10', 'D': '1101', 'E': '0010', 'F': '1111', 'G': '1110', 'H': '1100', 'I': '0000', 'J': '0110', 'K': '110011', 'L': '111110', 'M': '110111', 'N': '11', 'O': '0000', 'P': '0011', 'Q': '1100', 'R': '000011', 'S': '000000', 'T': '111111', 'U': '000010', 'V': '00001111', 'W': '11111111', 'X': '11000000', 'Y': '11001100', 'Z': '1100000000', '0': '1011111111', '1': '0011111111', '2': '0000111111', '3': '0000001111', '4': '0000000011', '5': '0000000000', '6': '1000000000', '7': '1110000000', '8': '1111100000', '9': '1111111000', ',': '1011', '.': '00110011', '(': '1100000011', ')': '1100000000', '/': '00111100', '?': '1111101100', '-': '1100001100'}


In [8]:
rosicrucian_cipher_numpy = {key:np.array([int(x) for x in rosicrucian_cipher_dict[key]] + [0, 0])
                    for key in rosicrucian_cipher_dict.keys()}
subset = list(rosicrucian_cipher_numpy.keys())

In [9]:
id2key = subset
key2id = {}
for idx, s in enumerate(subset):
    key2id[s] = idx

X = [ttype(rosicrucian_cipher_numpy[k])for k in subset]
Y = torch.LongTensor(np.arange(0,len(X)))
print(Y.max())
print(X, Y)

tensor(42)
[tensor([0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 0., 0.], device='cuda:0'), tensor([1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 0., 0.], device='cuda:0'), tensor([0., 0., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 1., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 1., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 1., 1., 1., 1., 0., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 1., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0.], device='cuda:0'), tensor([0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 0., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 0., 0.], device='cuda:0'), tensor([0., 0., 0., 0., 1., 1., 0., 0.], device='cuda:0'), tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([1.,

<ipython-input-9-326f2fb4fd85>:6: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  X = [ttype(rosicrucian_cipher_numpy[k])for k in subset]


In [10]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

# Three Layers

In [11]:
model = TCN(1, 43, [25, 25], kernel_size=45, dropout=0.0).cuda()
tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)


Total Weights: 86868
TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(1, 25, kernel_size=(45,), stride=(1,), padding=(44,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.0, inplace=False)
        (conv2): Conv1d(25, 25, kernel_size=(45,), stride=(1,), padding=(44,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.0, inplace=False)
        (net): Sequential(
          (0): Conv1d(1, 25, kernel_size=(45,), stride=(1,), padding=(44,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.0, inplace=False)
          (4): Conv1d(25, 25, kernel_size=(45,), stride=(1,), padding=(44,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.0, inplace=False)
        )
        (downsample): Conv1d(1, 25, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(25, 25, kern

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [12]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 5000
Trainscale = 10
device='cuda'
batch_size = 8
batches = int(np.ceil(43 / batch_size))

progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
times_100 = 0

for epoch_idx in progress_bar:
    perfs = []
    losses = []
    model.train()
    for batch_idx in range(batches):
        optimizer.zero_grad()
        loss = 0
        permute = np.arange(0, 43)
        for i in range(0, int(min(len(X) - (batch_idx*batch_size),
                              batch_size))
                       ):
            iv = X[permute[batch_idx*batch_size + i]]
            iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
            iv = iv.repeat(1,1,1,Trainscale)
            iv = iv.reshape(1,1,-1)
            tv = Y[permute[batch_idx*batch_size + i]].to(device)
            out = model(iv)
            loss += loss_func(out,
                         torch.cuda.LongTensor([tv]))
            perfs.append((torch.argmax(out, dim=-1) ==
                          tv).sum().item())


        loss = loss / min(len(X) - (batch_idx*batch_size),
                          batch_size)

        loss.backward()
        optimizer.step()


        #perfs = perfs[int(-loss_buffer_size/batch_size):]
        losses.append(loss.detach().cpu().numpy())
        #losses = losses[int(-loss_buffer_size/batch_size):]


        s = "{}:{:2} Loss: {:.4f}, Perf: {:.4f}"
        format_list = [epoch_idx, batch_idx, np.mean(losses),
                       np.sum(perfs)/((len(perfs)))]
        s = s.format(*format_list)
        progress_bar.set_description(s)
    if (np.sum(perfs)/((len(perfs))) == 1.0) & (np.mean(losses) < .11):
        times_100 += 1
        if times_100 >= 3:
            break


  0%|     | 0/5000 [00:00<?, ?it/s]     

KeyboardInterrupt: ignored

In [13]:
model.eval()
evald = []
evaldDict = {'test_perf':[],
             'rate':[]}
for nr in [1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40]:
#for nr in range(1,20):
    perfs = []
    for batch_idx, iv in enumerate(X):
        iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
        iv = iv.repeat(1,1,1,nr)
        iv = iv.reshape(1,1,-1)
        tv = Y[batch_idx].to(device)
        out = model(iv)
        loss = loss_func(out,
                         torch.cuda.LongTensor([tv]))


        perfs.append((torch.argmax(out, dim=-1) ==
                      tv).sum().item())
        #print(torch.argmax(out, dim=-1),
        #              tv)
    evaldDict['test_perf'].append(sum(perfs)/len(perfs))
    evaldDict['rate'].append(nr)
    print(nr, sum(perfs)/len(perfs))
    evald.append({'scale':nr,
                  'perf':sum(perfs)/len(perfs)})
scale_perfs = pd.DataFrame(evald)
scale_perfs.to_pickle(join("perf", "tcn_rosicrucian_test.dill"))

1 0.023255813953488372
2 0.023255813953488372
5 0.0
6 0.0
7 0.0
8 0.046511627906976744
9 0.3953488372093023
10 0.9302325581395349
11 0.6046511627906976
12 0.16279069767441862
13 0.046511627906976744
14 0.046511627906976744
15 0.023255813953488372
16 0.023255813953488372
17 0.023255813953488372
18 0.023255813953488372
19 0.046511627906976744
20 0.046511627906976744
30 0.046511627906976744
40 0.023255813953488372
